# Install Library

In [4]:
!pip install paperflow==0.1.4

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Attempting uninstall: paperflow
    Found existing installation: paperflow 0.1.0
    Uninstalling paperflow-0.1.0:
      Successfully uninstalled paperflow-0.1.0


In [7]:
import paperflow
paperflow.__version__

'unknown'

In [2]:
from paperflow import PaperPipeline

pipeline = PaperPipeline()

# Search across multiple sources
results = pipeline.search(
    "transformer attention mechanism",
    sources=["arxiv", "semantic_scholar"],
    max_results=10
)

# Process a paper (download → extract → chunk)
paper = pipeline.process(results.papers[0])

print(f"Title: {paper.metadata.title}")
print(f"Sections: {len(paper.sections)}")
print(f"Chunks: {len(paper.chunks)}")

# Export for RAG
docs = paper.to_langchain_documents()

ModuleNotFoundError: No module named 'paperflow'

In [6]:
print(f"Version: {paperflow.__version__}")

NameError: name 'paperflow' is not defined